In [21]:
## Handle secrets either from env vars or streamlit manager
import streamlit as st
import os
api_key = os.getenv("LITELLM_KEY")
if api_key is None:
    api_key = st.secrets["LITELLM_KEY"]
cirrus_key = os.getenv("CIRRUS_KEY")
if cirrus_key is None:
    cirrus_key = st.secrets["CIRRUS_KEY"]        

In [22]:
import os
import requests
import zipfile

def download_and_unzip(url, output_dir):
    response = requests.get(url)
    zip_file_path = os.path.basename(url)
    with open(zip_file_path, 'wb') as f:
        f.write(response.content)
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(output_dir)
    os.remove(zip_file_path)

download_and_unzip("https://minio.carlboettiger.info/public-data/hwc.zip", "hwc")

In [23]:
import pathlib
from langchain_community.document_loaders import PyPDFLoader

def pdf_loader(path):
    all_documents = []
    docs_dir = pathlib.Path(path)
    for file in docs_dir.iterdir():
        loader = PyPDFLoader(file)
        documents = loader.load()
        all_documents.extend(documents)
    return all_documents

docs = pdf_loader('hwc/')

In [24]:
# NRP embedding model tends to throw errors
# embedding = OpenAIEmbeddings(model = "embed-mistral", api_key = api_key, base_url = "https://llm.nrp-nautilus.io")

In [25]:
## Use the model on Cirrus instead:

from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(
                 model = "cirrus",
                 api_key =  cirrus_key, 
                 base_url = "https://llm.cirrus.carlboettiger.info/v1",
)

In [26]:
# Build a retrival agent
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=500)
splits = text_splitter.split_documents(docs)

In [27]:
# slow part here, runs on remote GPU
from langchain_core.vectorstores import InMemoryVectorStore
vectorstore = InMemoryVectorStore.from_documents(documents = splits, embedding = embedding)
retriever = vectorstore.as_retriever()

In [28]:
# Choose any of the models listed by their short-name:
# see `curl -H "Authorization: Bearer $OPENAI_API_KEY" https://llm.nrp-nautilus.io/v1/models`
"""
models = {"llama3": "llama3-sdsc", 
          "deepseek-small": "DeepSeek-R1-Distill-Qwen-32B",
          "deepseek": "deepseek-r1-qwen-qualcomm",
          "gemma3": "gemma3",
          "phi3": "phi3",
          "olmo": "olmo"
         }
"""
from langchain_openai import ChatOpenAI
#llm = ChatOpenAI(model = models['gemma3'],
                 #api_key = api_key, 
                 #base_url = "https://llm.nrp-nautilus.io",  
                 #temperature=0)


from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following scientific articles as the retrieved context to answer "
    "the question. Appropriately cite the articles from the context on which your answer is based using (Author, Year) format. "
    "Do not attempt to cite articles that are not in the context."
    "If you don't know the answer, say that you don't know."
    "Use up to five sentences maximum and keep the answer concise.\n\n"
    "{context}"
)
"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(retriever, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)
"""

#call the retriever by ourselves
def test_retriever_only(query: str, k: int = 5):
    retriever.search_kwargs["k"] = k  
    retrieved_docs = retriever.invoke(query)
    
    print(f"\n Query: {query}")
    print(f"\n Top {k} Retrieved Documents:\n" + "-"*60)
    
    for i, doc in enumerate(retrieved_docs):
        print(f"\n--- Document #{i+1} ---")
        print(doc.page_content[:1000])  
        if hasattr(doc, "metadata") and doc.metadata:
            print("\n[Metadata]:", doc.metadata)

In [29]:
test_query = "I live in Tanzania and am having issues with lions breaking into my boma and preying on cattle. What are a few ways to help me prevent this from happening in the future? Can you check these pdfs to see which ones might help?"
test_retriever_only(test_query, k=5)
test_retriever_only


 Query: I live in Tanzania and am having issues with lions breaking into my boma and preying on cattle. What are a few ways to help me prevent this from happening in the future? Can you check these pdfs to see which ones might help?

 Top 5 Retrieved Documents:
------------------------------------------------------------

--- Document #1 ---
hypothesis was that index of pre- and post-ban use
was the same for both types of feeding sites.
We calculated the average annual number of
observed bears during pre- and post-ban periods
for each feeding site. We then used these averages
instead of raw data from individual counts. We
calculated an annual index of feeding site use by
pooling data from all feeding sites (average number
of bears counted at feeding sites in post-ban period
divided by average number of bears counted in pre-
ban period). Subsequently, we used bootstrapping
with 1,000 simulations to test for differences between
SUPPLEMENTAL FEEDING AND BEAR DEPREDATIONS N Kavcˇicˇ et al

<function __main__.test_retriever_only(query: str, k: int = 5)>

In [30]:
test_query = "What should I do if elephants are destroying my crops? And what are the most cost-effective prevention methods, if there are any you know of? Can you check these pdfs to see which ones might help?"
test_retriever_only(test_query, k=5)
test_retriever_only


 Query: What should I do if elephants are destroying my crops? And what are the most cost-effective prevention methods, if there are any you know of? Can you check these pdfs to see which ones might help?

 Top 5 Retrieved Documents:
------------------------------------------------------------

--- Document #1 ---
hypothesis was that index of pre- and post-ban use
was the same for both types of feeding sites.
We calculated the average annual number of
observed bears during pre- and post-ban periods
for each feeding site. We then used these averages
instead of raw data from individual counts. We
calculated an annual index of feeding site use by
pooling data from all feeding sites (average number
of bears counted at feeding sites in post-ban period
divided by average number of bears counted in pre-
ban period). Subsequently, we used bootstrapping
with 1,000 simulations to test for differences between
SUPPLEMENTAL FEEDING AND BEAR DEPREDATIONS N Kavcˇicˇ et al. 113
Ursus 24(2):111–119 (2

<function __main__.test_retriever_only(query: str, k: int = 5)>

In [31]:
test_query = "I know jaguars can prey on goats and cattle, which I have; what measures can I take to save them from getting harmed? Can you check these pdfs to see which ones might help? "
test_retriever_only(test_query, k=5)
test_retriever_only


 Query: I know jaguars can prey on goats and cattle, which I have; what measures can I take to save them from getting harmed? Can you check these pdfs to see which ones might help? 

 Top 5 Retrieved Documents:
------------------------------------------------------------

--- Document #1 ---
458 SELECTIVE REMOVAL OF COYOTES * Blejwas et al. J. Wildl. Manage. 66(2):2002 
 was monitored, predation resumed only after the 
 new pair had formed (K. M. Blejwas, unpublished 
 data). These data suggest that loss of a mate and 
 the associated process of forming a new pair 
 bond alters the behavior of the surviving breeder, 
 temporarily interrupting predation on sheep. 
 Why Breeding Coyotes Kill Sheep 
 It is not surprising that breeding pairs are 
 responsible for most depredations of ewes and 
 large lambs. Most observations of coyotes attack- 
 ing large ungulate prey indicate that these are 
 cooperative endeavors involving more than 1 coy- 
 ote (Cahalane 1947, Robinson 1952, Bowyer 
 

<function __main__.test_retriever_only(query: str, k: int = 5)>

In [32]:
test_query = "I am trying to prevent coyotes from eating the calves of my free-range cattle. What may work best and incentivize them to stay away? Can you check these pdfs to see which ones might help? https://minio.carlboettiger.info/public-data/hwc.zip"
test_retriever_only(test_query, k=5)
test_retriever_only



 Query: I am trying to prevent coyotes from eating the calves of my free-range cattle. What may work best and incentivize them to stay away? Can you check these pdfs to see which ones might help? https://minio.carlboettiger.info/public-data/hwc.zip

 Top 5 Retrieved Documents:
------------------------------------------------------------

--- Document #1 ---
242 Conﬂict Intervention Priorities
helps foster more effective collaboration (Game et al.
2013; Lute et al. 2018). Third, both the survey results
and feedback were consistent with recent scholarship
(Redpath et al. 2017) that highlights participatory and
stakeholder-first conflict interventions as best practice
and advocates multipronged (Hazzah et al. 2014) and
adaptive management strategies (Bunnefeld et al. 2017).
Education and awareness programs were often cited in
feedback as being necessary additions to any interven-
tions. However, given the failures of many awareness-
based conservation programs (Schultz 2011), a further
e

<function __main__.test_retriever_only(query: str, k: int = 5)>

In [33]:
test_query = "Deers keep destroying and takiing over our large agricultural fields. Is there anything I can try to prevent this that won’t break the bank? Can you check these pdfs to see which ones might help? https://minio.carlboettiger.info/public-data/hwc.zip"
test_retriever_only(test_query, k=5)
test_retriever_only


 Query: Deers keep destroying and takiing over our large agricultural fields. Is there anything I can try to prevent this that won’t break the bank? Can you check these pdfs to see which ones might help? https://minio.carlboettiger.info/public-data/hwc.zip

 Top 5 Retrieved Documents:
------------------------------------------------------------

--- Document #1 ---
242 Conﬂict Intervention Priorities
helps foster more effective collaboration (Game et al.
2013; Lute et al. 2018). Third, both the survey results
and feedback were consistent with recent scholarship
(Redpath et al. 2017) that highlights participatory and
stakeholder-first conflict interventions as best practice
and advocates multipronged (Hazzah et al. 2014) and
adaptive management strategies (Bunnefeld et al. 2017).
Education and awareness programs were often cited in
feedback as being necessary additions to any interven-
tions. However, given the failures of many awareness-
based conservation programs (Schultz 2011), a f

<function __main__.test_retriever_only(query: str, k: int = 5)>

In [34]:
test_query = "We live in a suburb and bears sometimes come into our town to eat from our fruit trees and trash. What are the best ways for us to prevent this as a community without removing our fruit trees? Can you check these pdfs to see which ones might help? https://minio.carlboettiger.info/public-data/hwc.zip"
test_retriever_only(test_query, k=5)
test_retriever_only


 Query: We live in a suburb and bears sometimes come into our town to eat from our fruit trees and trash. What are the best ways for us to prevent this as a community without removing our fruit trees? Can you check these pdfs to see which ones might help? https://minio.carlboettiger.info/public-data/hwc.zip

 Top 5 Retrieved Documents:
------------------------------------------------------------

--- Document #1 ---
242 Conﬂict Intervention Priorities
helps foster more effective collaboration (Game et al.
2013; Lute et al. 2018). Third, both the survey results
and feedback were consistent with recent scholarship
(Redpath et al. 2017) that highlights participatory and
stakeholder-first conflict interventions as best practice
and advocates multipronged (Hazzah et al. 2014) and
adaptive management strategies (Bunnefeld et al. 2017).
Education and awareness programs were often cited in
feedback as being necessary additions to any interven-
tions. However, given the failures of many awaren

<function __main__.test_retriever_only(query: str, k: int = 5)>

In [35]:
test_query = "If we live in an area with a lot of wolves, what cattle husbandry strategies should I employ to prevent any sort of wildlife-human conflict? Can you check these pdfs to see which ones might help? https://minio.carlboettiger.info/public-data/hwc.zip"
test_retriever_only(test_query, k=5)
test_retriever_only


 Query: If we live in an area with a lot of wolves, what cattle husbandry strategies should I employ to prevent any sort of wildlife-human conflict? Can you check these pdfs to see which ones might help? https://minio.carlboettiger.info/public-data/hwc.zip

 Top 5 Retrieved Documents:
------------------------------------------------------------

--- Document #1 ---
242 Conﬂict Intervention Priorities
helps foster more effective collaboration (Game et al.
2013; Lute et al. 2018). Third, both the survey results
and feedback were consistent with recent scholarship
(Redpath et al. 2017) that highlights participatory and
stakeholder-first conflict interventions as best practice
and advocates multipronged (Hazzah et al. 2014) and
adaptive management strategies (Bunnefeld et al. 2017).
Education and awareness programs were often cited in
feedback as being necessary additions to any interven-
tions. However, given the failures of many awareness-
based conservation programs (Schultz 2011), a f

<function __main__.test_retriever_only(query: str, k: int = 5)>